In [3]:
import numpy as np
import os
import sys
from PIL import Image
from matplotlib.pyplot import figure
import pandas
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import torch
from torch import nn
import torch.optim as optim 
from torch.utils.data import TensorDataset, DataLoader

class MLP(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(256, 128),
      nn.Sigmoid(),
      nn.Linear(128, 128),
      nn.Sigmoid(),
      nn.Linear(128, 256),
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

ImageVectors = []

for i in range(10):
    path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),'characters1', str(i) +'.png')
    im = Image.open(path, 'r')
    gray = im.convert('L')
    bw = gray.point(lambda x: 0 if x<135 else 1, '1')
    ImageVectors.append(np.array(list(bw.getdata())))
    
for i in range(26):
    x = i + 65
    path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),'characters1', chr(i + 65) +'.png')
    im = Image.open(path, 'r')
    gray = im.convert('L')
    bw = gray.point(lambda x: 0 if x<135 else 1, '1')
    ImageVectors.append(np.array(list(bw.getdata())))

set1 = np.array(ImageVectors)

ImageVectors = []

for i in range(10):
    path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),'characters2', str(i) +'.png')
    im = Image.open(path, 'r')
    gray = im.convert('L')
    bw = gray.point(lambda x: 0 if x<135 else 1, '1')
    ImageVectors.append(np.array(list(bw.getdata())))
    
for i in range(26):
    x = i + 65
    path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),'characters2', chr(i + 65) +'.png')
    im = Image.open(path, 'r')
    gray = im.convert('L')
    bw = gray.point(lambda x: 0 if x<135 else 1, '1')
    ImageVectors.append(np.array(list(bw.getdata())))
    
set2 = np.array(ImageVectors)

imageTensor = torch.Tensor(set1)
_dataSet = TensorDataset(imageTensor, imageTensor)
_dataLoader = DataLoader(_dataSet)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(5):
    losses = []

    for batch_idx, (data, targets) in enumerate(_dataLoader):

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    
def calculate_performance_metrics(inputVector, outputVector):
    totalBlackPixelCount = sum(x == 0 for x in inputVector)
    totalWhitePixelCount = sum(x == 1 for x in inputVector)
    wrongBlackPixelCount = 0
    rightBlackPixelCount = 0
    for i in range(256):
        if outputVector[i] < 0.0001:
            if  abs(outputVector[i] - inputVector[i]) < 0.0001:
                rightBlackPixelCount += 1
            else:
                wrongBlackPixelCount += 1
    fh = rightBlackPixelCount/totalBlackPixelCount
    ffa = wrongBlackPixelCount/totalWhitePixelCount
    return fh, ffa 

model.eval()
print("Checking accuracy on Training Set")
for i in range(36):
    output = model(torch.from_numpy(set1[i].astype('float32'))).detach().numpy()
    for j in range(256):
        if output[j] > 0:
            output[j] = 1
        else:
            output[j] = 0
    fh, ffa = calculate_performance_metrics(set1[i], output)
    print("FH:", fh, "FFA:", ffa)

print("Checking accuracy on Test Set")
for i in range(36):
    output = model(torch.from_numpy(set2[i].astype('float32'))).detach().numpy()
    for j in range(256):
        if output[j] > 0:
            output[j] = 1
        else:
            output[j] = 0
    fh, ffa = calculate_performance_metrics(set2[i], output)
    print("FH:", fh, "FFA:", ffa)

Checking accuracy on Training Set
FH: 0.8648648648648649 FFA: 0.32413793103448274
FH: 0.8160919540229885 FFA: 0.4260355029585799
FH: 0.659217877094972 FFA: 0.35064935064935066
FH: 0.7134502923976608 FFA: 0.23529411764705882
FH: 0.7025316455696202 FFA: 0.336734693877551
FH: 0.7559523809523809 FFA: 0.17045454545454544
FH: 0.7976878612716763 FFA: 0.0963855421686747
FH: 0.7518248175182481 FFA: 0.33613445378151263
FH: 0.711340206185567 FFA: 0.11290322580645161
FH: 0.7005347593582888 FFA: 0.15942028985507245
FH: 0.6912751677852349 FFA: 0.3644859813084112
FH: 0.746031746031746 FFA: 0.04477611940298507
FH: 0.7133757961783439 FFA: 0.35353535353535354
FH: 0.6631578947368421 FFA: 0.30303030303030304
FH: 0.6146788990825688 FFA: 0.21052631578947367
FH: 0.6020942408376964 FFA: 0.4153846153846154
FH: 0.7248677248677249 FFA: 0.13432835820895522
FH: 0.5706521739130435 FFA: 0.5416666666666666
FH: 0.8090909090909091 FFA: 0.363013698630137
FH: 0.71875 FFA: 0.3828125
FH: 0.6201117318435754 FFA: 0.415584415

'\nprint("Checking accuracy on Training Set")\nfor x, y in _dataSet:\n    test = x\n    break\n\noutput = model(x).detach().numpy()\nprint(np.array(x))\n\nfor i in range(256):\n    if output[i] > 0:\n        output[i] = 1\n    else:\n        output[i] = 0\nprint(output)\n\nprint(torch.from_numpy(set2[0].astype(\'float32\')))\ntest = model(torch.from_numpy(set2[0].astype(\'float32\'))).detach().numpy()\nprint(test)\n'